# 1. import necessaries library for setting up the environmet

In [24]:
#main library for data analysis
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#library for json file
import json 
#convert address to latitude, longitude values 
from geopy.geocoders import Nominatim
#library for handle request
import requests
#transform json file into pandas df
from pandas.io.json import json_normalize
#matplotlib for plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors
#slkearn for KMeans Cluster
from sklearn.cluster import KMeans
#folium for mapping 
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



# 2. web scrapping 

In [3]:
#import Beautifulsoup for scrapping wiki
from bs4 import BeautifulSoup

In [4]:
#store wiki page in url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#open url with beautiful soup
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
#print(soup)

#### if you print soup you get text in html so we needed to scrape them into table with 3 columns
#### Create a [ ] so we can stored data within 3 cloumns 

In [6]:
#create an empty list to stored 
postalCode_list = [] 
borough_list = [] 
neighborhood_list = []
#create for loop to find tag<tr> and tag<td>
for i in soup.find('table').find_all('tr'):
    x = i.find_all('td')
    if (len(x) > 0):
        postalCode_list.append(x[0].text)
        borough_list.append(x[1].text)
        neighborhood_list.append(x[2].text.rstrip('\n'))
print( 'This is postalCode_list',postalCode_list[0:5])
print('This is borough_list',borough_list[0:5])
print('This is neihborhood_list',neighborhood_list[0:5])

This is postalCode_list ['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
This is borough_list ['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
This is neihborhood_list ['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront']


#### Let's put these in DataFrame

In [7]:
df = [('PostalCode', postalCode_list), ('Borough', borough_list),('Neighborhood', 
       neighborhood_list)]

In [8]:
df1 = pd.DataFrame.from_dict(dict(df))
df1.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### As we can see there are 'Not assigned' in Borough, Neighborhood columns, we don't want to deal with missing valued

In [9]:
df1_dropna = df1[df1['Borough'] != 'Not assigned'].reset_index(drop = True)
df1_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### From the instruction,
#### 'More than one neighborhood can exist in one postal code area. 
#### For example, in the table on the Wikipedia page, 
#### you will notice that M5A is listed twice and has two neighborhoods: 
#### Harbourfront and Regent Park. These two rows will be combined 
#### into one row with the neighborhoods separated with a comma 
#### as shown in row 11 in the above table
##### let's grouped neighbood by Postal and Borough

In [42]:
df1_dropna_grouped = df1_dropna.groupby(['PostalCode','Borough'],
as_index = False).agg(lambda x: ','.join(x))
df_toronto = df1_dropna_grouped
df_toronto.loc[80:85]

,PostalCode,Borough,Neighborhood
80,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
81,M6N,York,"The Junction North,Runnymede"
82,M6P,West Toronto,"High Park,The Junction South"
83,M6R,West Toronto,"Parkdale,Roncesvalles"
84,M6S,West Toronto,"Runnymede,Swansea"
85,M7A,Queen's Park,Not assigned


#### According to the instruction,
#### 'If a cell has a borough but a Not assigned neighborhood, 
#### then the neighborhood will be the same as the borough. 
#### So for the 9th cell in the table on the Wikipedia page, 
#### the value of the Borough and the Neighborhood columns will be Queen's Park.'
#### 85th has 'Not assigned' valued in Neighborhood, so we decied to assign <br><br> the value to be the same as Borough

In [20]:
neighborhood_na = df1_dropna_grouped['Neighborhood'] == 'Not assigned'
#located the 'Not assigned' and assign to the same value as Borough
df1_dropna_grouped.loc[neighborhood_na, 'Neighborhood'] = df1_dropna_grouped.loc[neighborhood_na, 'Borough']
df1_dropna_grouped.loc[80:85]

,PostalCode,Borough,Neighborhood
80,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
81,M6N,York,"The Junction North,Runnymede"
82,M6P,West Toronto,"High Park,The Junction South"
83,M6R,West Toronto,"Parkdale,Roncesvalles"
84,M6S,West Toronto,"Runnymede,Swansea"
85,M7A,Queen's Park,Queen's Park


In [21]:
Cleaned_df = df1_dropna_grouped
Cleaned_df.shape

(103, 3)

# 3. Foursquare API For Coordinates 

#### After we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [35]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('download completed!')

download completed!


In [40]:
coor = pd.read_csv('Geospatial_data.csv')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### As you can see in coor DataFrame has the same column as our df_toronto DataFrame so let's merge them together to get a full df 

In [120]:
Cleaned_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [123]:
#First we indexes by df.PostalCode = coor.Postal Code and inner join(intersect)
df_toronto_1 = Cleaned_df.set_index('PostalCode')
coor_1 = coor.set_index('Postal Code')
df_toronto_full = pd.concat([df_toronto_1, coor_1], axis = 1, join = 'inner')
df_toronto_full.reset_index(inplace = True)
df_toronto_full.rename(columns = {'index':'PostalCode'}, inplace = True)
Toronto_df = df_toronto_full

In [127]:
Toronto_df.loc[1:11]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304
